In [1]:
import src.inference.helpers as ih
import src.models.builders as mb
import src.models.cpts.helpers as cpth
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import src.modelling_ar.ar as model_ar

In [2]:
(
    model,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
) = mb.o2sat_fev1_point_in_time_model_shared_healthy_vars(160, 30, "Male")

In [3]:
cpt = cpth.get_cpt([ecFEV1, HFEV1, AR])

In [8]:
cpt_for_ecfev1 = cpt[ecFEV1.get_bin_for_value(1)[1], :, :]

ar_cpt = AR.cpt
# ar_cpt = 100000 + AR.midbins * AR.midbins * AR.midbins
# ar_cpt = AR.midbins + 50
# ar_cpt = np.exp(AR.midbins)
# ar_cpt = np.zeros(len(AR.midbins))
# ar_cpt[AR.get_bin_for_value(10)[1]] = 1

# ar_cpt = ar_fit
ar_cpt = ar_cpt / sum(ar_cpt)

hfev1 = np.matmul(cpt_for_ecfev1, ar_cpt) 

fig = make_subplots(rows=2, cols=1)
ih.plot_histogram(fig, AR, ar_cpt, AR.a, AR.b, 1, 1)
ih.plot_histogram(fig, HFEV1, hfev1, HFEV1.a, HFEV1.b, 2, 1)
fig.show()

In [10]:
cpt_hfev1 = hfev1
cpt_hfev1 = np.ones(len(HFEV1.midbins)) / len(HFEV1.midbins)
# cpt_hfev1 = HFEV1.midbins

ar = np.matmul(cpt_hfev1, cpt_for_ecfev1)

fig = make_subplots(rows=2, cols=1)
ih.plot_histogram(fig, HFEV1, cpt_hfev1, HFEV1.a, HFEV1.b, 1, 1)
ih.plot_histogram(fig, AR, ar, AR.a, AR.b, 2, 1)
fig.show()

In [14]:
fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=AR.midbins, y=ar, mode="lines+markers", name="from hfev1"))
ar_tmp = np.log(AR.midbins) / sum(np.log(AR.midbins))
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="log fit"))
ar_tmp = AR.midbins * AR.midbins * AR.midbins + 100000
ar_tmp = ar_tmp / sum(ar_tmp)
fig.add_trace(
    go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="square fit")
)
# Polyfit
ar_tmp = model_ar.get_prior_for_uniform_hfev1_message(AR)
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_tmp, mode="lines+markers", name="poly fit"))
fig.show()

In [210]:
# Fit a polynomial function to the data in ar
for deg in range(3, 13):
    print("degree", deg)
    p = np.polyfit(AR.midbins[:-4], ar[:-4], deg)
    ar_fit = np.polyval(p, AR.midbins)

    print("residual error", sum(np.abs(ar[:-4] - ar_fit[:-4])))

# Plot the original data and the fitted data
fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=AR.midbins, y=ar, mode="lines+markers", name="data"))
fig.add_trace(go.Scatter(x=AR.midbins, y=ar_fit, mode="lines+markers", name="fit"))
fig.show()

degree 3
residual error 0.024801388647439623
degree 4
residual error 0.009967995840924511
degree 5
residual error 0.004022692327867009
degree 6
residual error 0.001630324111333209
degree 7
residual error 0.0006463233699155746
degree 8
residual error 0.0002515372150580751
degree 9
residual error 9.781780929920929e-05
degree 10
residual error 3.66148457972091e-05
degree 11
residual error 1.4054224854977843e-05
degree 12
residual error 5.39941279047336e-06


In [211]:
p

array([ 6.84808411e-23, -3.01603307e-20,  5.86472319e-18, -6.60296098e-16,
        4.75372724e-14, -2.28184316e-12,  7.39699647e-11, -1.60333825e-09,
        2.27139833e-08, -1.85804099e-07,  1.94333647e-06,  9.79127935e-05,
        1.00017236e-02])

# Is the prior uniform in log space?

In [85]:
ar_prior = model_ar.get_prior_for_uniform_hfev1_message(AR)

log_prior = np.log(1-ar_prior)
log_prior = log_prior / sum(log_prior)

fig = make_subplots(rows=1, cols=1)
# Add scatter plot
fig.add_trace(go.Scatter(x=100-AR.midbins, y=ar_prior, mode="lines+markers"))
# Add x axis title
fig.update_xaxes(title_text="1-AR: capacity to let air through")
fig.show()
